# Wine Quality Model
This notebook trains and packages for deployment a linear model that predicts wine quality based on physio-chemical properties.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.datasets import load_wine
import pandas as pd
import numpy as np
from WineQualityModelService import WineQualityModelService
import joblib

In [2]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

Download dataset

In [3]:
# Read the wine-quality csv file from the URL
data, y = load_wine(as_frame=True, return_X_y=True)
data["quality"] = y

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

In [8]:
# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

alpha = 0.5
l1_ratio = 0.5

In [12]:
print(list(test_x.iloc[0,:]))
print(test_y.iloc[0,:].to_json())

[13.9, 1.68, 2.12, 16.0, 101.0, 3.1, 3.39, 0.21, 2.14, 6.1, 0.91, 3.33, 985.0]
{"quality":0}


Fit linear model

In [11]:
lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
lr.fit(train_x, train_y)

predicted_qualities = lr.predict(test_x)

(rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
print("  RMSE: %s" % rmse)
print("  MAE: %s" % mae)
print("  R2: %s" % r2)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.374367150612527
  MAE: 0.2952075902077537
  R2: 0.755340262067303


Package with BentoML

In [12]:
bento_lr = WineQualityModelService()
bento_lr.pack("model", lr)
bento_lr.save()

[2021-05-05 16:11:42,634] INFO - BentoService bundle 'WineQualityModelService:20210505161141_54093C' saved to: /home/dlowl/bentoml/repository/WineQualityModelService/20210505161141_54093C


'/home/dlowl/bentoml/repository/WineQualityModelService/20210505161141_54093C'

In [13]:
with open('model.joblib', 'wb') as f:
    joblib.dump(lr, f)